# Web Scraping

In [1]:
import urllib.request, urllib.parse, urllib.error
import requests
import re
import csv
import ssl
import collections
import itertools
import sqlite3
from math import*
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 

In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
#we need to use a user agent here, because metacritics does not accept bots
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [3]:
#remove all html symbols and get text
def tag_removal(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean,"", text)

In [4]:
def combine(list1,list2):#combines 2 list
    return [x + [y[1]] for (x, y) in zip(list1, list2)]

# Imdb

In [5]:
def Imdb_Names():
    url_imdb = 'http://www.imdb.com/chart/top' #url for top Imdb movies
    response = requests.get(url_imdb)
    soup = BeautifulSoup(response.text, 'lxml')
    movies = soup.select('td.titleColumn')#reference to movie title
    top_movies = movies[:50] #here we get the top 50 movies we want
    imdb = []
    n = 0
    for index in range(0, len(top_movies)):
        n = n + 1
        movie_detail = movies[index].get_text()
        #get movie index, title and year in text format
        movie = (' '.join(movie_detail.split()).replace('.', ''))#remove period after index
        movie_title = movie[len(str(index))+1:-7]#removes year at the end
        data = [n,movie_title]#list of index and movie title
        imdb.append(data) #appends to empty list imdb
    return imdb  
        

def Imdb_Links():
    #now we need to click into each movie name in order to see detailed information
    #so we need to extract all the urls
    url_imdb = 'https://www.imdb.com/chart/top'#url for top Imdb movies
    response = requests.get(url_imdb)
    soup = BeautifulSoup(response.text, 'lxml')
    links = soup.select('td.titleColumn a')#reference to movie link
    top_50_links = links[:50]
    imdb_movie_links = []
    for a in top_50_links:
        hyperlink = a.attrs.get('href')
        links = 'https://www.imdb.com'+hyperlink#combine to get complete url
        imdb_movie_links.append(links)
    return imdb_movie_links #returns a list of all the movie links   

def Fetch_Genre_For_Dict():
    movie_genre = []
    n = 0
    for url in Imdb_Links():
        n = n + 1
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        ref_genre = soup.find_all('div', class_ = 'see-more inline canwrap')[1]#reference to movie genre
        ref_genre_text = ref_genre.find_all('a')#reference to genre text
        for i in range(0,len(ref_genre_text)):
            genre = tag_removal(str(ref_genre_text[i]).replace(' ','').lower())
            #cleaning and replace space with no space as well as convert to lowercase
            movie_genre.append(genre)#append genre into the empty genre list
    return movie_genre    
 

def Imdb_Dict(my_list): 
    count = {} #empty dictionary
    for j in my_list: 
        count[j] = count.get(j, 0) + 1 
        #if element not present add the count as 0 else increment count by 1
    return count

def Fetch_Genre_CSV():
    movie_genre = []
    n = 0
    for url in Imdb_Links():
        n = n + 1
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        #we are using the html parser in bs4 to read all the html files on the webpage
        ref_genre = soup.find_all('div', class_ = 'see-more inline canwrap')[1]#reference to movie genre
        ref_genre_text = ref_genre.find_all('a')#reference to genre text
        ref_genre_remove_tag = tag_removal(str(ref_genre_text).strip())
        #remove square brackets
        ref_genre_remove_brackets = (' '.join(ref_genre_remove_tag.split()).replace('[', '').replace(']', ''))
        data = [n,ref_genre_remove_brackets]#list of index and movie genre
        movie_genre.append(data)
    return movie_genre    

 

def IMDB():
    imdb = []
    imdb = combine(Imdb_Names(),Fetch_Genre_CSV())#combine movie names and genre's fetched in a list
    return imdb

def Imdb_Csv():    
    filename = "Top50_Imdb_movie.csv"
    headers = ['number' ,'name', 'genre']
    with open(filename,'w',newline='') as f:  
        w = csv.writer(f, headers)  
        w.writerow(headers)
        w.writerows(IMDB())
    
    
    print("Imdb_CSV written!")


# Rotten_Tomatoes

In [6]:
def Rotten_Tomatoes_Names():
    rotten_tomatoes = []
    n = 0 
    url_rotten_tomatoes = 'https://www.rottentomatoes.com/top/bestofrt/'
    response = requests.get(url_rotten_tomatoes)
    soup = BeautifulSoup(response.text, 'lxml')
    movies = soup.select('table.table a')
    final_movies = movies[:50]
    for index in range(0, len(final_movies)): 
        n = n + 1
        movie_string = movies[index].get_text()
        movie = (' '.join(movie_string.split()).replace('.', ''))
        data = [n,movie]
        rotten_tomatoes.append(data)
    return rotten_tomatoes  
   

def Rotten_Tomatoes_Links():
    n = 0
    url_rotten_tomatoes = 'https://www.rottentomatoes.com/top/bestofrt/'
    response = requests.get(url_rotten_tomatoes)
    soup = BeautifulSoup(response.text, 'lxml')
    movie_links = soup.select('table.table a')
    imdb_top_50_links = movie_links[:50]
    movie_links = []
    for a in imdb_top_50_links:
        n = n + 1
        links = a.attrs.get('href')
        url = 'https://www.rottentomatoes.com'+links
        movie_links.append(url)
    return movie_links    

def Genre_Dict():
    movie_genre = []
    n = 0
    for url in Rotten_Tomatoes_Links():
        n = n + 1
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        ref_genre_text = soup.find('div', class_ = 'meta-value genre').text
        cleaned_genre = ' '.join(ref_genre_text).replace(',','').replace(' ','').replace('and','\n')
        genre2 = cleaned_genre.split()
        movie_genre.append(genre2)
    return movie_genre    

def Rotten_Tomatoes_Dict(passed_list): 
    count = {} 
    for i in passed_list: 
        for x in i:
            count[x] = count.get(x, 0) + 1
    return count 

def Rotten_Tomatoes_Genre_Csv():
    movie_genre = []
    n = 0
    for url in Rotten_Tomatoes_Links():
        n = n + 1
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        ref_genre_text = soup.find('div', class_ = 'meta-value genre').text
        genre_cleaned = (' '.join(str(ref_genre_text).split()).replace('[', '').replace(']', '').replace('and', ',').replace('Genre(s):',''))
        cleaned_text = [n,genre_cleaned]
        movie_genre.append(cleaned_text)
    return movie_genre    



def Rotten_Tomatoes():
    rotten_tomatoes = []
    rotten_tomatoes = combine(Rotten_Tomatoes_Names(),Rotten_Tomatoes_Genre_Csv())
    return rotten_tomatoes

def Rotten_Tomatoes_Csv():    
    filename = "Rotten_Tomatoes_top_50.csv"
    headers = ['number', 'name', 'genre']
    with open(filename,'w',newline='') as f:  
        csvwriter = csv.writer(f, headers) 
        
        #csvwriter writes all the headers and combined movie list with name and genres
        csvwriter.writerow(headers)  
        csvwriter.writerows(Rotten_Tomatoes()) 
 
    print("Rotten_Tomatoes_CSV written!")

# Metacritic

In [7]:
def Metacritics_Names():
    n = 0
    url = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc"
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    container = soup.find_all('td', class_ = 'clamp-summary-wrap')
    movie_names = []
    movies = container[:50] #here we get the top 50 movies we want
    for movie in movies:
        n = n + 1
        name = movie.find('h3').text
        data = [n, name]
        movie_names.append(data)
    return movie_names

def Metacritics_Links():
    n = 0
    url = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc"
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    container = soup.find_all('td', class_ = 'clamp-summary-wrap')
    movie_links = []
    movies = container[:50]
    for movie in movies:
        n = n + 1
        tag = movie.find('a', class_ = 'title')
        link = tag.get('href', None)
        url = "https://www.metacritic.com"+link
        movie_links.append(url)
    return movie_links

def Metacritics_Genre_Dict():
    metacritic_genre_dict = []
    n = 0
    for url in Metacritics_Links():
        n = n + 1
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        ref_genre_text = soup.find('div', class_ = 'genres').text
        genre = ref_genre_text.replace('Genre(s):','').replace(' ','').replace('\n','').replace(',','\n').lower().split() 
        metacritic_genre_dict.append(genre)

    return metacritic_genre_dict    

def Metacritic_Dictionary(list1): 
    count = {} 
    for i in list1: 
        for x in i:
            count[x] = count.get(x, 0) + 1
    return count 

def Metacritics_Genre_Csv():
    metacritic_genre_csv = []
    n = 0
    for url in Metacritics_Links():
        n = n + 1
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        ref_genre = soup.find('div', class_ = 'genres').text
        ref_genre_clean = (' '.join(str(ref_genre).split()).replace('[', '').replace(']', '').replace('and', ',').replace('Genre(s):',''))
        cleaned_text = [n,ref_genre_clean]
        metacritic_genre_csv.append(cleaned_text)
    return metacritic_genre_csv    

        
def Metacritics():
    metactric = []
    metactric = combine(Metacritics_Names(),Metacritics_Genre_Csv())
    return metactric   

def Metacritics_Csv():    
    filename = "Metactric_top_50.csv"
    #create csv which has headers number,name and genre
    headers = ['number', 'name', 'genre']
    with open(filename,'w',newline='') as f:  
        csvwriter = csv.writer(f, headers) 
        
        #csvwriter writes all the headers and combined movie list with name and genres
        csvwriter.writerow(headers)  
        csvwriter.writerows(Metacritics())    
    print("Metacritics_CSV written!")
    

In [8]:
def Cosine_Similarity_between_movie_genre():
    
    imdb = Imdb_Dict(Fetch_Genre_For_Dict())
    rottentomatoes = Rotten_Tomatoes_Dict(Genre_Dict())
    metacritic = Metacritic_Dictionary(Metacritics_Genre_Dict())

    union_imdb_rottentomatoes = dict(list(imdb.items()) + list(rottentomatoes.items()))
    #Fetch values from imdb and rotten tomatoes dictionary and then union both the lists into a dictionary
    union_rottentomatoes_metactritic = dict(list(rottentomatoes.items()) + list(metacritic.items()))
    #Fetch values from rotten tomatoes and metacritic dictionary and then union both the lists into a dictionary
    print(union_imdb_rottentomatoes)
    print(union_rottentomatoes_metactritic)
    union_sets={'set1': union_imdb_rottentomatoes,'set2': union_rottentomatoes_metactritic}
    #store both the sets of the union result in a genre dictionary

    def cosine_similarity(x,y):
        i1 = {}
        i2 = {}
        v2 = []
        v1 =[]

        if len(x) > len(y):
            i1 = x
            i2 = y
        else:
            i1 = y
            i2 = x
            
        # Getting vector from dictionary values
        v1 = list(i1.values())
        for k in i1.keys():    
            if k in i2:
                v2.append(float(i2[k]))
            else :
                v2.append(float(0))
        #operations based on cosine formula
        numerator = sum(a*b for a,b in zip(v2,v1))
        denominator = round(sqrt(sum([a*a for a in v1])),3)*round(sqrt(sum([a*a for a in v2])),3)
        return round(numerator/float(denominator),3)

    print("The 3 Websites have a cosine similarity of : ")
    print (cosine_similarity(union_sets['set1'],union_sets['set2']))


In [10]:
def db_connection():
    con = sqlite3.connect("Top50.db")#this opens the Top50 database, if it does not exist, it will create a new one
#it can also connect to remote databases, but we are just doing it on our local server
    if(con):
        print("Established Connection Successfully")#if connection is successful then print success
    else:
        print("Connection Error")#print error if connection is not possible    
    cur = con.cursor() #cursor reads the database, like open() when we are dealing with files
    
    cur.execute('DROP TABLE IF EXISTS IMDB')#drop table if IMDB table already exists
    cur.execute("CREATE TABLE IMDB (number, name, genre)")#create a table IMDB which has 3columns i.e. number,name and genre
 
    with open('Top50_Imdb_movie.csv','r') as file_input:# open csv in read mode
        dict_read = csv.DictReader(file_input) 
        db_inp = [(i['number'],i['name'], i['genre'])  for i in dict_read]
     #for all values read by csv create columns of number, name and genre and give as input to the database   
    insert_intodb = cur.executemany("INSERT INTO IMDB ( number, name, genre) VALUES ( ?, ?, ?);", db_inp)
    #Insert query into the IMDB table which takes all the values
    cur.execute('DROP TABLE IF EXISTS Rotten_Tomatoes')
    #drop table if Rotten_Tomatoes table already exists
    cur.execute("CREATE TABLE Rotten_Tomatoes (number, name, genre)")
    #create a table Rotten_Tomatoes which has 3columns
    with open('Rotten_Tomatoes_top_50.csv','r') as file_input:
      # open csv in read mode
        dict_read = csv.DictReader(file_input) 
        db_inp = [(i['number'],i['name'], i['genre'])  for i in dict_read]
    #for all values read by csv create columns of number, name and genre and give as input to the database
    insert_intodb  = cur.executemany("INSERT INTO Rotten_Tomatoes ( number, name, genre) VALUES ( ?, ?, ?);", db_inp)
    #Insert query into the Rotten_Tomatoes table which takes all the values
    cur.execute('DROP TABLE IF EXISTS Metacritic')
     #drop table if Metacritic table already exists
    cur.execute("CREATE TABLE Metacritic (number, name, genre)")
    #create a table Metacritic which has 3columns
    with open('Metactric_top_50.csv','r') as file_input:
      # open csv in read mode
        dict_read = csv.DictReader(file_input) 
        db_inp = [(i['number'],i['name'], i['genre'])  for i in dict_read]
    #for all values read by csv create columns of number, name and genre and give as input to the database
    insert_intodb  = cur.executemany("INSERT INTO Metacritic ( number, name, genre) VALUES ( ?, ?, ?);", db_inp)
    #Insert query into the Metacritic table which takes all the values
    if(insert_intodb ):
        print("Successfully added")
    else:
        print("Error")    
    con.commit() #we need to commit it to save it, just like office files
    con.close()

In [11]:
if __name__ == "__main__":
    Imdb_Csv()
    Rotten_Tomatoes_Csv()
    Metacritics_Csv()
    db_connection()
    Cosine_Similarity_between_movie_genre()

Imdb_CSV written!
Rotten_Tomatoes_CSV written!
Metacritics_CSV written!
Established Connection Successfully
Successfully added
{'drama': 21, 'crime': 5, 'action': 10, 'thriller': 7, 'biography': 5, 'history': 2, 'adventure': 14, 'fantasy': 13, 'western': 2, 'romance': 4, 'sci-fi': 8, 'mystery': 7, 'comedy': 18, 'war': 2, 'family': 8, 'animation': 5, 'musical': 2, 'music': 2, 'horror': 4, 'scifi': 5, 'kids': 8}
{'fantasy': 3, 'action': 3, 'adventure': 3, 'scifi': 5, 'horror': 3, 'mystery': 10, 'thriller': 13, 'kids': 8, 'family': 3, 'animation': 3, 'comedy': 9, 'drama': 39, 'musical': 3, 'crime': 4, 'music': 1, 'romance': 11, 'war': 5, 'history': 5, 'film-noir': 4, 'western': 2, 'sport': 1, 'documentary': 1, 'biography': 2}
The 3 Websites have a cosine similarity of : 
0.79
